In [605]:
import os
from os import environ

import requests
import io

import datetime

import numpy as np
import pandas as pd

from pandas import Series
from pandas import DataFrame

import matplotlib.pyplot as plt

In [606]:
#route = environ['route']
destination='ACE'
path='/home/ubuntu/Data/'
downweight_granularity=['FltNum','Direction','dtime','dday','month']

In [607]:
def map_interval(x):
    if x>=350:
        return '350'
    elif x>=300:
        return '300'
    elif x>=250:
        return '250'
    elif x>=200:
        return '200'
    elif x>=150:
        return '150'
    elif x>=100:
        return '100'
    else:
        return '0-99'

# Data Import

In [608]:
Data=pd.read_csv(path+'SQLsnippets_tours/SQLsnippet_'+destination+'_allprod.csv',sep=',')
Data['calendar_date']=pd.to_datetime(Data['calendar_date'])
Data['pax_total']=Data['pax_total'].fillna(0)
Data['group_pax']=Data['group_pax'].fillna(0)
#Capa=pd.read_csv(path+'SQLsnippets_tours/SQLsnippet_'+destination+'_capa.csv',sep=',')
#Capa['calendar_date']=pd.to_datetime(Capa['calendar_date'])

Data=Data[[x for x in Data.columns if x!='dprio']+['dprio']]

AllFlights=Data[['calendar_date','dep_time','flight_number','airport_arrival','airport_depart']].drop_duplicates()
AllFlights.index=range(AllFlights.shape[0])
AllTime=[x for x in range(0,367)]

AllTime=Series(AllTime*AllFlights.shape[0])

AllFlights=AllFlights.loc[np.repeat(AllFlights.index,len(AllTime)/AllFlights.shape[0]),:]
AllFlights.index=range(AllFlights.shape[0])

AllFlights=pd.concat([AllFlights,Series([0]*len(AllTime)),Series([0]*len(AllTime)),AllTime],ignore_index=True,axis=1)

AllFlights.columns=Data.columns

Data=pd.concat([Data,AllFlights])

Data=Data.drop_duplicates(subset=['calendar_date','flight_number','dep_time','dprio'])

Data=Data.groupby(['calendar_date','airport_depart','airport_arrival','dprio'])[['pax_total','group_pax']].sum().reset_index()

Data['diff']=(Data['calendar_date']-pd.to_datetime(datetime.datetime.today().strftime('%Y-%m-%d'))).apply(lambda x: x.days)

Data=Data.loc[Data['dprio']>Data['diff'],:]

Data=Data[[x for x in Data.columns if x!='diff']]

Data=Data[['calendar_date','airport_depart','airport_arrival','dprio','pax_total','group_pax']]\
.sort_values(by=['calendar_date','dprio'],ascending=[True,False])

Data=Data.set_index(['calendar_date','airport_depart','airport_arrival','dprio'])

Data=Data.groupby(level=[0,1,2]).diff().reset_index()
Data=Data.dropna(how='any')

In [631]:
Holiday_LUX=pd.read_csv('/home/ubuntu/Data/holiday/LUX-LUX_special_days.csv',encoding="ISO-8859-1")
Holiday_SCN=pd.read_csv('/home/ubuntu/Data/holiday/LUX-SCN_special_days.csv',encoding="ISO-8859-1")
Holiday_SXB=pd.read_csv('/home/ubuntu/Data/holiday/LUX-SXB_special_days.csv',encoding="ISO-8859-1")
Holiday_BRU=pd.read_csv('/home/ubuntu/Data/holiday/LUX-BRU_special_days.csv',encoding="ISO-8859-1")

In [604]:
pd.to_datetime('2020-07-23')-pd.to_datetime('2019-07-26')

Timedelta('363 days 00:00:00')

In [ ]:
Data=Data.groupby(['calendar_date','airport_depart','airport_arrival','dprio'])[['pax_total','group_pax']].sum().reset_index()

In [ ]:
Data['pax_total'].unique()

In [ ]:


Data=Data.loc[Data['calendar_date']<=pd.to_datetime('2019-07-26'),:]

Data['dprio_min']=Data.groupby(['calendar_date','flight_number','dep_time'])['dprio'].transform('min')
Data['dprio_max']=Data.groupby(['calendar_date','flight_number','dep_time'])['dprio'].transform('max')
Data['sum_pax']=Data.groupby(['calendar_date','flight_number','dep_time'])['pax_total'].transform('sum')

Data['n_flight']=Data.groupby(['calendar_date','dprio','airport_arrival','airport_depart'])['dep_time'].transform('size')

In [ ]:
Data.loc[(Data['dprio']==0)&(Data['sum_pax']>0)&(Data['n_flight']==2),:]

In [ ]:
Data['n_flight']

In [ ]:
Data.loc[(Data['calendar_date']=='2017-03-30')&(Data['airport_arrival']=='ACE')&(Data['airport_depart']=='LUX')\
         &(Data['dprio']==232),:].sort_values(by='dprio')

In [ ]:
Data.loc[(Data['calendar_date']=='2017-03-30')&(Data['airport_arrival']=='ACE')&(Data['airport_depart']=='LUX'),:].sort_values(by='dprio')

In [ ]:
Data=Data.merge(Data.loc[(Data['dprio']==Data['dprio_min'])&(Data['pax_total']>0),['calendar_date','flight_number','dep_time']],
on=['calendar_date','flight_number','dep_time'])

In [ ]:
Data.shape

In [ ]:
Data.groupby(['n_flight'])['calendar_date'].size()

In [ ]:
Data.groupby(['calendar_date','flight_number','dep_time'])['n_flight'].apply(lambda x: len(set(x)))

In [ ]:
Data.loc[(Data['calendar_date']=='2019-07-21')&(Data['flight_number']=='LG-0742')&(Data['n_flight']==1),:].sort_values('dprio')

In [ ]:
Data[['calendar_date','flight_number','dep_time']].drop_duplicates().shape

In [ ]:
Data.loc[Data['dprio_min']!=0,['calendar_date','flight_number','dep_time']].drop_duplicates().shape

In [ ]:
Data.loc[(Data['dprio_min']==0)&(Data['sum_pax']==0),['calendar_date','flight_number','dep_time']].drop_duplicates().shape

In [ ]:
Data=Data.loc[(Data['dprio_min']==0)&(Data['sum_pax']>0),:]
Data[['calendar_date','flight_number','dep_time']].drop_duplicates().shape

In [ ]:
Data=Data[['calendar_date','flight_number','dep_time','dprio_max']]\
.drop_duplicates(subset=['calendar_date','flight_number','dep_time'])
Data['Interval']=Data['dprio_max'].transform(lambda x: map_interval(x))

In [ ]:
Data.groupby('Interval')['Interval'].size()

In [ ]:
Data=Data.loc[Data['dprio_min']==0,:]

In [ ]:
Data.shape

In [ ]:
Data=Data.merge(Capa[['calendar_date','dprio','flight_number','dep_time','capacity']],
on=['calendar_date','dprio','flight_number','dep_time'],how='left')

In [ ]:
Data.shape

In [ ]:
Data.loc[(Data['calendar_date']==pd.to_datetime('2018-12-16'))&(Data['flight_number']=='LG-0750'),:].sort_values(by='dprio')

In [ ]:
test=Data.loc[pd.isnull(Data['capacity']),:]

In [ ]:
test['dprio_min_2']=test.groupby(['calendar_date','flight_number'])['dprio'].transform('min')

In [ ]:
test.loc[test['dprio_min_2']==40,:]

In [ ]:
Data.loc[Data['dprio_min']==24,'flight_number'].unique()

In [ ]:
Data.loc[(Data['dprio_min']==24)&(Data['flight_number']=='LG-0741'),:]

In [ ]:
test.loc[pd.isnull(test['capacity']),:]

In [ ]:
test.loc[pd.isnull(test['capacity']),['calendar_date','dprio']].drop_duplicates()['calendar_date'].unique()

In [ ]:
test.loc[(test['calendar_date']=='2017-01-01')&(test['flight_number']=='LG-0183'),:].sort_values('dprio')

In [ ]:
test.loc[pd.isnull(test['capacity']),:]

In [ ]:


Data.columns=['extraction_date','calendar_date','dep_time','flight_number','airport_arrival',
              'airport_depart','pax_total','group_pax','cabin_capacity']

Data=Data.dropna(how='any')
Data.index=range(Data.shape[0])
Data=Data[['airport_depart','airport_arrival','flight_number','dep_time','extraction_date', 'calendar_date',
     'pax_total','group_pax','cabin_capacity']]
Data=Data.rename(columns={'airport_depart': 'SegFrom', 'airport_arrival': 'SegTo', 'flight_number': 'FltNum', 
'dep_time': 'DepTime', 'extraction_date': 'IssueDate', 'calendar_date': 'DepDate',
'pax_total': 'NumPax', 'group_pax': 'Group pax', 'cabin_capacity': 'Cabin Capacity'})

Data['IssueDate']=Data['IssueDate'].apply(lambda x: str(x)[6:8]+'/'+str(x)[4:6]+'/'+str(x)[0:4])
Data['DepDate']=Data['DepDate'].apply(lambda x: str(x)[6:8]+'/'+str(x)[4:6]+'/'+str(x)[0:4])

# CHANGE FORMAT

Data=Data.loc[Data['FltNum'].apply(lambda x: len(x))==7,:]
Data['FltNum']=Data['FltNum'].apply(lambda x: str(str(x).split('-')[1]))
Data['NumPax']=Data['NumPax'].apply(lambda x: float(str(x).split(' ')[0]))
Data['Group pax']=Data['Group pax'].apply(lambda x: float(str(x).split(' ')[0]))
Data['Cabin Capacity']=Data['Cabin Capacity'].apply(lambda x: float(str(x).split(' ')[0]))
Data['DepDayMonth']=Data['DepDate'].apply(lambda x: str(x).split('/')[0]+'/'+str(x).split('/')[1])
Data['DepMonth']=Data['DepDate'].apply(lambda x: str(x).split('/')[1]+'/'+str(x).split('/')[2])

Data=Data.loc[Data['DepTime'].apply(lambda x: len(str(x))>=5),:]

# MAP DESTINATION COMBINATION TO INBOUND OR OUTBOUND

Map_AirportsToDirection=DataFrame([[route.split('-')[0],route.split('-')[1]],
[route.split('-')[1],route.split('-')[0]],['O','I']]).transpose()
Map_AirportsToDirection.columns=['SegFrom','SegTo','Direction']
Data=Data.merge(Map_AirportsToDirection,on=['SegTo','SegFrom'])

### Change format of remaining columns

In [ ]:
IssueDate=Data['IssueDate'].unique().tolist()
IssueDate_Map=DataFrame([IssueDate,IssueDate]).transpose()
IssueDate_Map.columns=['IssueDate','NewFormat']
IssueDate_Map['NewFormat']=IssueDate_Map['NewFormat'].apply(lambda x: pd.to_datetime(x.split(' ')[0],format='%d/%m/%Y'))

Data=Data.merge(IssueDate_Map,on=['IssueDate'])
Data=Data[[x for x in Data.columns if x!='IssueDate']]
Data=Data.rename(columns={'NewFormat': 'IssueDate'})

#######################################

DepartureDate=Data['DepDate'].unique().tolist()
DepartureDate_Map=DataFrame([DepartureDate,DepartureDate]).transpose()
DepartureDate_Map.columns=['DepDate','NewFormat']
DepartureDate_Map['NewFormat']=DepartureDate_Map['NewFormat'].apply(lambda x: pd.to_datetime(x.split(' ')[0],format='%d/%m/%Y'))
DepartureDate_Map['yday']=DepartureDate_Map['NewFormat'].apply(lambda x: x.timetuple().tm_yday)

DepartureDate_Map['dday_num']=DepartureDate_Map['NewFormat'].apply(lambda x: x.weekday())
Weekday_Map=DataFrame([[x for x in range(0,8)],['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']]).transpose()
Weekday_Map.columns=['dday_num','dday']
Weekday_Map['dday_num']=Weekday_Map['dday_num'].astype('int')

DepartureDate_Map=DepartureDate_Map.merge(Weekday_Map,on='dday_num')
DepartureDate_Map=DepartureDate_Map[[x for x in DepartureDate_Map if x!='dday_num']]
DepartureDate_Map=DepartureDate_Map.sort_values(by='NewFormat')

Data=Data.merge(DepartureDate_Map,on=['DepDate'])
Data=Data[[x for x in Data.columns if x!='DepDate']]
Data=Data.rename(columns={'NewFormat': 'DepDate'})

#######################################

Data['DepTime']=Data['DepTime'].astype('str')
Data['DepTime']=Data['DepTime'].apply(lambda x: '0'+x[0]+':'+x[1:3]+':00' if len(x)==5 else x[0:2]+':'+x[2:4]+':00')
DepartureTime=Data['DepTime'].unique().tolist()
DepartureTime_Map=DataFrame([DepartureTime,[pd.to_datetime(x,format='%H:%M:%S') for x in DepartureTime]]).transpose()
DepartureTime_Map.columns=['DepTime','NewFormat']
DepartureTime_Map['dtime']=DepartureTime_Map['NewFormat'].apply(lambda x: (x-datetime.datetime.combine(x.date(),datetime.time(0,0))).seconds/60)

Data=Data.merge(DepartureTime_Map,on=['DepTime'])
Data=Data[[x for x in Data.columns if x!='DepTime']]
Data=Data.rename(columns={'NewFormat': 'DepTime'})

#######################################

Data['month']=Data['DepDate'].apply(lambda x: int(x.month))

#######################################

Data['Dprio']=(Data['DepDate']-Data['IssueDate']).apply(lambda x: float(x.days))
Data=Data.loc[(Data['Dprio']>0)&(Data['Dprio']<=366)&(Data['NumPax']>0)&(~pd.isnull(Data['DepTime'])),:]

Data=Data.sort_values(by=['DepDate','FltNum','dtime','Dprio'],ascending=[True,True,True,False])

# CAPACITY SMOOTHING

#Data.loc[Data['Cabin Capacity']>70,'Cabin Capacity']=67

In [ ]:
# Data=Data.sort_values(by=[])
# Data=Data.drop_duplicates(subset=['extraction_date','calendar_date','flight_number','airport_arrival','airport_depart','dep_time'])

### Deduct single departure time

In [ ]:
DprioMin=Data.groupby(['DepDate','FltNum'])['Dprio'].min().reset_index()
DprioMin=DprioMin.rename(columns={'Dprio': 'Dprio_min'})

Data=Data.merge(DprioMin,on=['DepDate','FltNum'])

NumPaxMax=Data.loc[Data['Dprio']==Data['Dprio_min'],:].groupby(['FltNum','DepDate'])['NumPax'].max().reset_index()
NumPaxMax=NumPaxMax.rename(columns={'NumPax': 'NumPax_max'})

Data=Data.merge(NumPaxMax,on=['DepDate','FltNum'])

Map_FltnumDeptime=Data.loc[(Data['Dprio']==Data['Dprio_min'])&(Data['NumPax']==Data['NumPax_max']),['DepDate','FltNum','dtime']].drop_duplicates()

Map_FltnumDeptime=Map_FltnumDeptime.rename(columns={'dtime': 'dtime_new'})

Data=Data.merge(Map_FltnumDeptime,on=['DepDate','FltNum'])

Data['dtime']=Data['dtime_new']
Data=Data[[x for x in Data.columns if x!='dtime_new' and x!='Dprio_min' and x!='NumPax_max']]

# Cabin Capacity

In [ ]:
Capacity=Data[['DepDate','FltNum','dtime','Dprio','Cabin Capacity']].drop_duplicates().copy()

intermediate=Capacity.groupby(['DepDate','FltNum','dtime'])['Dprio'].min().reset_index().rename(columns={'Dprio': 'Dprio_min'})
intermediate['DepDate']=intermediate['DepDate'].apply(lambda x: pd.to_datetime(x))
Capacity['DepDate']=Capacity['DepDate'].apply(lambda x: pd.to_datetime(x))

Capacity_smooth=Capacity.merge(intermediate[[x for x in intermediate.columns if x!='Cabin Capacity']],
on=['DepDate','FltNum','dtime'])
Capacity_smooth=Capacity_smooth.loc[Capacity_smooth['Dprio']==Capacity_smooth['Dprio_min'],[x for x in Capacity_smooth.columns if 'Dprio' not in x]]

Dropdown_comb=Data[['DepDate','FltNum','dtime']].drop_duplicates()
Dropdown_comb.index=range(Dropdown_comb.shape[0])
Dprio_all=Data['Dprio'].unique().tolist()

Dropdown_comb=Dropdown_comb.loc[np.repeat(Dropdown_comb.index,len(Dprio_all)),:]
Dropdown_comb.index=range(Dropdown_comb.shape[0])

Dropdown_comb=pd.concat([Dropdown_comb,Series(Dprio_all*Dropdown_comb.drop_duplicates().shape[0])],axis=1)
Dropdown_comb=Dropdown_comb.rename(columns={0: 'Dprio'})

Dropdown_comb['DepDate']=Dropdown_comb['DepDate'].apply(lambda x: pd.to_datetime(x))
Capacity['DepDate']=Capacity['DepDate'].apply(lambda x: pd.to_datetime(x))

Dropdown_comb=Dropdown_comb.merge(Capacity,on=['DepDate','FltNum','dtime','Dprio'],how='left')
Dropdown_comb=Dropdown_comb.merge(Capacity_smooth.rename(columns={'Cabin Capacity': 'C_smooth'}),on=['DepDate','FltNum','dtime'])

Dropdown_comb['Cabin Capacity']=Dropdown_comb[['Cabin Capacity','C_smooth']].apply(lambda x: x[1] if pd.isnull(x[0]) else x[0],axis=1)

Dropdown_comb=Dropdown_comb[[x for x in Dropdown_comb.columns if x!='C_smooth']]

deptime=[str(x/60).split('.')[0]+':'+str(round(float('0.'+str(x/60).split('.')[1])*60)) for x in Dropdown_comb['dtime'].unique()]
deptime=[x+'0' if len(x.split(':')[1])==1 else x for x in deptime]
deptime=['0'+x if len(x.split(':')[0])==1 else x for x in deptime]

Map_DepTime=DataFrame([Capacity['dtime'].unique(),[x+'0' if len(x.split(':')[1])==1 else x for x in deptime]]).transpose()
Map_DepTime.columns=['dtime','deptime']
Map_DepTime['dtime']=Map_DepTime['dtime'].astype('int')

Dropdown_comb=Dropdown_comb.merge(Map_DepTime,on='dtime')
Dropdown_comb['dtime']=Dropdown_comb['deptime']
Dropdown_comb=Dropdown_comb[[x for x in Dropdown_comb.columns if x!='deptime']]

# Pax Data

In [ ]:
Pax=Data.groupby(['DepDate','Dprio','FltNum','dtime','yday','Direction','month','dday'])['NumPax'].sum().reset_index()
Pax['Dprio_min']=Pax.groupby(['DepDate','FltNum','dtime','yday','Direction','month','dday'])['Dprio'].transform('min')

#####################################################################################

Data_NoObs=Pax.loc[:,['DepDate','FltNum','dtime','yday','Direction','month','dday']].drop_duplicates().copy()

distinct_flights=Data_NoObs.shape[0]
Data_NoObs.index=range(Data_NoObs.shape[0])
Data_NoObs=Data_NoObs.iloc[np.repeat([x for x in Data_NoObs.index],365),:]

Data_NoObs['Dprio']=[x for x in range(1,366)]*distinct_flights
Data_NoObs['NumPax']=0

Data_NoObs=Data_NoObs.merge(Pax[['DepDate','FltNum','dtime','yday','Direction','month','dday','Dprio_min']].drop_duplicates(),
                           on=['DepDate','FltNum','dtime','yday','Direction','month','dday'])

Data_NoObs=Data_NoObs.loc[Data_NoObs['Dprio']>=Data_NoObs['Dprio_min'],:]
Pax=pd.concat([Pax,Data_NoObs])
Pax=Pax.drop_duplicates(subset=['DepDate','Dprio','FltNum','dtime','yday','Direction','month','dday'])
Pax=Pax[[x for x in Pax.columns if '_min' not in x]]
Data_NoObs=Data_NoObs[[x for x in Data_NoObs.columns if '_min' not in x]]

#####################################################################################

Pax=Pax.sort_values(by=['DepDate','FltNum','dtime','Dprio'],ascending=[True,True,True,False])
Pax=Pax.set_index(['DepDate','Dprio','FltNum','dtime','yday','Direction','month','dday'])

Pax=Pax.groupby(level=[0,2,3,4,5,6,7]).diff().reset_index()
Pax=Pax.dropna(how='any')

#####################################################################################

Data_NoObs=Pax.loc[:,['DepDate','FltNum','dtime','yday','Direction','month','dday']].drop_duplicates().copy()

distinct_flights=Data_NoObs.shape[0]
Data_NoObs.index=range(Data_NoObs.shape[0])
Data_NoObs=Data_NoObs.iloc[np.repeat([x for x in Data_NoObs.index],365),:]

Data_NoObs['Dprio']=[x for x in range(1,366)]*distinct_flights
Data_NoObs['NumPax']=0

Pax=pd.concat([Pax,Data_NoObs])
Pax=Pax.drop_duplicates(subset=['DepDate','Dprio','FltNum','dtime','yday','Direction','month','dday'])

Pax=Pax.sort_values(by=['DepDate','FltNum','dtime','Dprio'],ascending=[True,True,True,False])
Pax.index=range(Pax.shape[0])

# Group bookings

In [ ]:
GroupPax=Data.groupby(['DepDate','Dprio','FltNum','dtime','yday','Direction','month','dday'])['Group pax'].sum().reset_index()
GroupPax['Dprio_min']=GroupPax.groupby(['DepDate','FltNum','dtime','yday','Direction','month','dday'])['Dprio'].transform('min')

#####################################################################################

Data_NoObs=GroupPax.loc[:,['DepDate','FltNum','dtime','yday','Direction','month','dday']].drop_duplicates().copy()

distinct_flights=Data_NoObs.shape[0]
Data_NoObs.index=range(Data_NoObs.shape[0])
Data_NoObs=Data_NoObs.iloc[np.repeat([x for x in Data_NoObs.index],365),:]

Data_NoObs['Dprio']=[x for x in range(1,366)]*distinct_flights
Data_NoObs['Group pax']=0

Data_NoObs=Data_NoObs.merge(GroupPax[['DepDate','FltNum','dtime','yday','Direction','month','dday','Dprio_min']].drop_duplicates(),
                           on=['DepDate','FltNum','dtime','yday','Direction','month','dday'])

Data_NoObs=Data_NoObs.loc[Data_NoObs['Dprio']>=Data_NoObs['Dprio_min'],:]
GroupPax=pd.concat([GroupPax,Data_NoObs])
GroupPax=GroupPax.drop_duplicates(subset=['DepDate','Dprio','FltNum','dtime','yday','Direction','month','dday'])
GroupPax=GroupPax[[x for x in GroupPax.columns if '_min' not in x]]
Data_NoObs=Data_NoObs[[x for x in Data_NoObs.columns if '_min' not in x]]

#####################################################################################

GroupPax=GroupPax.sort_values(by=['DepDate','FltNum','dtime','Dprio'],ascending=[True,True,True,False])
GroupPax=GroupPax.set_index(['DepDate','Dprio','FltNum','dtime','yday','Direction','month','dday'])

GroupPax=GroupPax.groupby(level=[0,2,3,4,5,6]).diff().reset_index()
GroupPax=GroupPax.dropna(how='any')

#####################################################################################

Data_NoObs=GroupPax.loc[:,['DepDate','FltNum','dtime','yday','Direction','month','dday']].drop_duplicates().copy()

distinct_flights=Data_NoObs.shape[0]
Data_NoObs.index=range(Data_NoObs.shape[0])
Data_NoObs=Data_NoObs.iloc[np.repeat([x for x in Data_NoObs.index],365),:]

Data_NoObs['Dprio']=[x for x in range(1,366)]*distinct_flights
Data_NoObs['Group pax']=0

Data_NoObs=Data_NoObs[GroupPax.columns]

GroupPax=pd.concat([GroupPax,Data_NoObs])
GroupPax=GroupPax.drop_duplicates(subset=['DepDate','Dprio','FltNum','dtime','yday','Direction','month','dday'])

GroupPax=GroupPax.sort_values(by=['DepDate','FltNum','dtime','Dprio'],ascending=[True,True,True,False])

GroupPax.index=range(GroupPax.shape[0])

#######################################################

GroupPax['Group_pax_cumul']=GroupPax.groupby(['DepDate','FltNum','yday','Direction','month','dday'])['Group pax'].transform(lambda x: np.cumsum(x))

# Remove group bookings from pax data and cabin capacity

In [ ]:
Pax=Pax.merge(GroupPax,on=['DepDate','Dprio','FltNum','dtime','yday','Direction','month','dday'])
Pax['NumPax']=Pax['NumPax']-Pax['Group pax']
Pax=Pax[[x for x in Pax.columns if x!='Group pax']]

##############################################################################

# deptime=[str(x/60).split('.')[0]+':'+str(round(float('0.'+str(x/60).split('.')[1])*60)) for x in GroupPax['dtime'].unique()]
# deptime=[x+'0' if len(x.split(':')[1])==1 else x for x in deptime]
# deptime=['0'+x if len(x.split(':')[0])==1 else x for x in deptime]

# Map_DepTime=DataFrame([Capacity['dtime'].unique(),[x+'0' if len(x.split(':')[1])==1 else x for x in deptime]]).transpose()
# Map_DepTime.columns=['dtime','deptime']
# Map_DepTime['dtime']=Map_DepTime['dtime'].astype('int')

# GroupPax=GroupPax.merge(Map_DepTime,on='dtime')
# GroupPax['dtime']=GroupPax['deptime']
# GroupPax=GroupPax[[x for x in GroupPax.columns if x!='deptime']]

# Dropdown_comb=Dropdown_comb.merge(GroupPax[['DepDate','FltNum','dtime','Dprio','Group_pax_cumul']],
# on=['DepDate','FltNum','dtime','Dprio'])
# Dropdown_comb['Cabin Capacity_removeGroup']=Dropdown_comb['Cabin Capacity']-Dropdown_comb['Group_pax_cumul']
# Dropdown_comb['Cabin Capacity_removeGroup']=Dropdown_comb[['Cabin Capacity_removeGroup', 'Cabin Capacity']].apply(lambda x: x[1] if x[0]>x[1] else x[0],axis=1)

# Dropdown_comb['Cabin Capacity_removeGroup']=Dropdown_comb['Cabin Capacity_removeGroup'].apply(lambda x: 0 if x<0 else x)
# Dropdown_comb['Cabin Capacity']=Dropdown_comb['Cabin Capacity_removeGroup']
# Dropdown_comb=Dropdown_comb[[x for x in Dropdown_comb.columns if x!='Group pax' and x!='Cabin Capacity_removeGroup']]

# Data export

### Pax

In [ ]:
TrainingSet_Pax=Pax.loc[(Pax['DepDate']>=pd.to_datetime('2018-04-01'))\
&(Pax['DepDate']<pd.to_datetime(datetime.datetime.today().strftime('%Y-%m-%d'))),].copy()
TestSet_Pax=Pax.loc[Pax['DepDate']>=pd.to_datetime(datetime.datetime.today().strftime('%Y-%m-%d')),].copy()

TrainingSet_Pax.to_csv(path+'Intermediate_Output/R_Training_Pax.csv',index=False)
TestSet_Pax['DepDate']=TestSet_Pax['DepDate'].apply(lambda x: pd.to_datetime(x))
TestSet_Pax.to_csv(path+'Intermediate_Output/R_Test_Pax.csv',index=False)

### Group Pax

In [ ]:
GroupPax['Route']=route
GroupPax=GroupPax.loc[GroupPax['DepDate']>=pd.to_datetime(datetime.datetime.today().strftime('%Y-%m-%d')),].copy()

deptime=[str(x/60).split('.')[0]+':'+str(round(float('0.'+str(x/60).split('.')[1])*60)) for x in GroupPax['dtime'].unique()]
deptime=[x+'0' if len(x.split(':')[1])==1 else x for x in deptime]
deptime=['0'+x if len(x.split(':')[0])==1 else x for x in deptime]

Map_DepTime=DataFrame([GroupPax['dtime'].unique(),[x+'0' if len(x.split(':')[1])==1 else x for x in deptime]]).transpose()
Map_DepTime.columns=['dtime','deptime']
Map_DepTime['dtime']=Map_DepTime['dtime'].astype('int')

GroupPax=GroupPax.merge(Map_DepTime,on='dtime')
GroupPax['dtime']=GroupPax['deptime']
GroupPax=GroupPax[[x for x in GroupPax.columns if x!='deptime']]

if route in ['LUX-DUB','LUX-CDG','LUX-VIE','LUX-HAM','SCN-HAM','LUX-BRI','LUX-VCE','LUX-PRG']:
    GroupPax.to_csv(path+'FrontEnd_Input_pierrick/GroupPax_'+route+'.csv',index=False)

elif route in ['LUX-LCY','LUX-CPH','LUX-GVA','LUX-LIS','LUX-OPO','LUX-NCE']:
    GroupPax.to_csv(path+'FrontEnd_Input_nancy/GroupPax_'+route+'.csv',index=False)
else:
    GroupPax.to_csv(path+'FrontEnd_Input_patrick/GroupPax_'+route+'.csv',index=False)

### Cabin Capacity

In [ ]:
Dropdown_comb['Route']=route
#Dropdown_comb=Dropdown_comb.loc[Dropdown_comb['DepDate']>=pd.to_datetime(datetime.datetime.today().strftime('%Y-%m-%d')),].copy()
Dropdown_comb.to_csv(path+'Intermediate_Output/Capacity_forPlots_'+route+'.csv',index=False)